In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator

In [7]:
data = spark.read.csv('./data/movielens_ratings.csv',inferSchema=True,header=True)

In [8]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [10]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [11]:
training, test = data.randomSplit([0.8,0.2])

In [12]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId',ratingCol='rating')

In [13]:
model = als.fit(training)

In [14]:
predictions = model.transform(test)

In [15]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      6|   1.0|    28|-0.40677965|
|      0|   1.0|    26| -1.1555917|
|      1|   1.0|    26| -0.2993824|
|      4|   1.0|    12|  1.6580071|
|      3|   2.0|    22| 0.45266414|
|      5|   2.0|    22|  1.2163234|
|      5|   1.0|    13|  2.0514534|
|      0|   1.0|     3|  1.4445819|
|      2|   1.0|     3| 0.14221323|
|      6|   2.0|    19| 0.64177036|
|      3|   1.0|     9|  1.8905009|
|      6|   1.0|     9| -0.1093989|
|      6|   1.0|    17|  1.4276503|
|      3|   2.0|     8|  1.2146015|
|      5|   1.0|     8|  1.6667974|
|      6|   2.0|    23|  1.6677667|
|      0|   3.0|    10| 0.29941705|
|      6|   3.0|    24| 0.23619089|
|      3|   1.0|    29| -1.0510795|
|      6|   1.0|    14|  1.5772221|
+-------+------+------+-----------+
only showing top 20 rows



In [16]:
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

In [17]:
rmse = evaluator.evaluate(predictions)

In [19]:
print('RMSE')
print(rmse)

RMSE
1.829506006274327


In [20]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [21]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     11|    11|
|     13|    11|
|     19|    11|
|     20|    11|
|     23|    11|
|     35|    11|
|     37|    11|
|     51|    11|
|     69|    11|
|     70|    11|
|     72|    11|
|     82|    11|
|     90|    11|
|     94|    11|
+-------+------+



In [22]:
recommendations = model.transform(single_user)

In [23]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     69|    11|  3.8354487|
|     11|    11|  3.0763936|
|     19|    11|  2.9181192|
|     13|    11|   2.795875|
|     90|    11|   2.732246|
|     72|    11|  1.6740944|
|     37|    11|   1.303528|
|     20|    11|  1.1530203|
|     82|    11| 0.83206403|
|     23|    11| 0.58731294|
|     51|    11|  0.2553258|
|     94|    11|-0.19854087|
|     70|    11|  -1.343159|
|      9|    11| -1.3819368|
|     35|    11| -1.6634449|
+-------+------+-----------+

